## Apply enrichment method

This notebook aims to show that SOPHIE can be easily extended to use multiple different enrichment methods. The enrichment methods were selected from [Geistlinger et. al. paper](https://academic.oup.com/bib/article/22/1/545/5722384). We selected a subset of the methods in the paper based on those that could 1) be applied to both RNA-seq and array data and 2)those that covered a wide range of statistical performances.

The methods we selected were (description of the methods are below):
* GSEA
* GSVA
* CAMERA
* ORA

These methods have a range of different characteristics:
* Runtime: GSEA has the longest runtime compared to CAMERA which has the shortest, GSVA and ORA are in the middle
* Type of method: GSVA is self-contained vs the others are competative
* Number of gene sets found to be statistically signficant: GSVA found the most (median ~70%), CAMERA found ~25% and ORA and GSEA found very few

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import pickle

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from ponyo import utils
from generic_expression_patterns_modules import ranking

np.random.seed(123)

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_human_general.tsv")
)

params = utils.read_config(config_filename)

In [3]:
# Load params
local_dir = params["local_dir"]
project_id = params["project_id"]
statistic = params["gsea_statistic"]
hallmark_DB_filename = params["pathway_DB_filename"]
num_runs = params["num_simulated"]
dataset_name = params["dataset_name"]

# Select enrichment method
# enrichment_method = ["GSEA", GSVA", "CAMERA", "ORA"]
# If enrichment_method == "GSEA" then use "padj" to rank
# If enrichment_method == "GSVA" then use "ES" to rank
# If enrichment_method == "CAMERA" then use "FDR" to rank
# If using "ORA" then use "p.adjust" to rank
enrichment_method = "ORA"
col_to_rank_pathways = params["rank_pathways_by"]

In [4]:
# Load DE stats directory
DE_stats_dir = os.path.join(local_dir, "DE_stats")

# Template experiment gene expression
template_expression_filename = os.path.join(base_dir, dataset_name, params["processed_template_filename"])

# Template experiment DE stats
template_DE_stats_filename = os.path.join(
    DE_stats_dir,
    f"DE_stats_template_data_{project_id}_real.txt"
)

# Metadata file with sample grouping to define comparison
metadata_filename = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "metadata",
    f"{project_id}_groups.tsv"
)

In [5]:
# Output file
pathway_summary_filename = os.path.join(
    base_dir, 
    dataset_name, 
    f"generic_pathway_summary_{project_id}_{enrichment_method}.tsv"
)

## Enrichment methods
These methods are important because they permit differential expression questions to be posed in terms of ensembles of genes representing pathways or other biologically interpretable processes. 

There are two main groups of methods: ‘self-contained’ gene set tests examine a set of genes in their own right without reference to other genes in the genome whereas ‘competitive’ gene set tests compare genes in the test set relative to all other genes. Self-contained tests are of interest for assessing the relevance of an individual biological process to the experiment at hand, whereas the competitive tests focus more on distinguishing the most important biological processes from those that are less important. Competitive tests are overwhelmingly more commonly used in the genomic literature. (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3458527/)

* [CAMERA](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3458527/) (Correlation Adjusted MEan RAnk gene set test) is based on the idea of estimating the variance inflation factor associated with inter-gene correlation, and incorporating this into parametric or rank-based test procedures. (available in limma) 
  * Competitive gene set test
  * Performs the same rank-based test procedure as GSEA, but also estimates the correlation between genes, instead of treating genes as independent
  * Recall GSEA: 1) Rank all genes using DE association statistics. 2) An enrichment score (ES) is defined as the maximum distance from the middle of the ranked list. Thus, the enrichment score indicates whether the genes contained in a gene set are clustered towards the beginning or the end of the ranked list (indicating a correlation with change in expression). 3) Estimate the statistical significance of the ES by a phenotypic-based permutation (permute samples assigned to label) test in order to produce a null distribution for the ES (i.e. scores based on permuted phenotype)
  * Appropriate for small and large fold changes
* [GSVA](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3618321/) (Gene Set Variation Analysis) calculates sample-wise gene set enrichment scores as a function of genes inside and outside the gene set. This method is well-suited for assessing gene set variation across a dichotomous phenotype. (biocontuctor package GSVA) 
  * Self-contained gene set test
  * Estimates variation of gene set enrichment over the samples independently of any class label
* [ORA](https://www.rdocumentation.org/packages/clusterProfiler/versions/3.0.4/topics/enricher) (over-representation analysis) uses the hypergeometric test to determine if there a significant over-representation of pathway in the selected set of DEGs. Here we're using clusterProfiler library but there are multiple options for this analysis. See [slide 6](https://docs.google.com/presentation/d/1t4rK7UiLAeIKIzeRJK-YzspNUfGM-8nuRCcevh2lx34/edit?usp=sharing)

In [6]:
# Create "<local_dir>/GSEA_stats/" subdirectory
os.makedirs(os.path.join(local_dir, "GSA_stats"), exist_ok=True)

In [7]:
# Load pathway data
hallmark_DB_filename = params["pathway_DB_filename"]

**Apply enrichment to template experiment**

See supplementary tables: https://academic.oup.com/bib/advance-article/doi/10.1093/bib/bbz158/5722384

In [8]:
%%R -i base_dir -i local_dir -i project_id -i template_expression_filename -i hallmark_DB_filename -i metadata_filename -i enrichment_method -o template_enriched_pathways

source(paste0(base_dir, '/generic_expression_patterns_modules/other_enrichment_methods.R'))

out_filename <- paste(local_dir, 
                      "GSA_stats/",
                      enrichment_method,
                      "_stats_template_data_",
                      project_id,
                      "_real.txt", 
                      sep = "")

if (enrichment_method == "GSVA"){
    
    template_enriched_pathways <- find_enriched_pathways_GSVA(
        template_expression_filename,
        hallmark_DB_filename
    )
}
else if (enrichment_method == "CAMERA"){
    
    template_enriched_pathways <- find_enriched_pathways_CAMERA(
        template_expression_filename,
        metadata_filename,
        hallmark_DB_filename
    )
}
else if (enrichment_method == "ORA"){
    
    template_enriched_pathways <- find_enriched_pathways_ORA(
        template_expression_filename,
        metadata_filename, 
        hallmark_DB_filename
    )
}
write.table(as.data.frame(template_enriched_pathways), file = out_filename, row.names = F, sep = "\t")

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: Rcpp

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘edgeR’


  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following object is masked from ‘package:DEFormats’:

    DGEList


  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: S4Vectors

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: stats4

  warnings.

[1] "Checking sample ordering..."
[1] TRUE


/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [9]:
# Quick check
print(template_enriched_pathways.shape)
template_enriched_pathways

(50, 8)


,NGenes,PropDown,PropUp,Direction,PValue,FDR,PValue.Mixed,FDR.Mixed
0,193,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
1,189,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
2,181,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
3,180,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
4,138,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
5,89,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
6,55,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
7,54,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
8,32,0.000000,1.000000,Up,0.0001,0.0001,0.0001,0.0001
9,185,0.000000,0.994595,Up,0.0001,0.0001,0.0001,0.0001


**Apply enrichment to simulated experiments**

Note: GSA takes a while to run (each simulated experiment took ~2-3 minutes)

In [10]:
%%R -i project_id -i local_dir -i hallmark_DB_filename -i metadata_filename -i num_runs -i base_dir -i enrichment_method

source(paste0(base_dir, '/generic_expression_patterns_modules/other_enrichment_methods.R'))

for (i in 0:(num_runs-1)){
    simulated_expression_filename <- paste(local_dir, 
                                           "pseudo_experiment/selected_simulated_data_",
                                           project_id,
                                           "_", 
                                           i,
                                           "_processed.txt",
                                           sep = "")

    out_filename <- paste(local_dir,
                          "GSA_stats/",
                          enrichment_method,
                          "_stats_simulated_data_",
                          project_id,
                          "_",
                          i,
                          ".txt", 
                          sep = "")
    
    if (enrichment_method == "GSVA"){
        enriched_pathways <- find_enriched_pathways_GSVA(
            simulated_expression_filename, 
            hallmark_DB_filename
        ) 
        write.table(as.data.frame(enriched_pathways), file = out_filename, row.names = F, sep = "\t")
        print("in GSVA")
    }
    else if (enrichment_method == "CAMERA"){
        enriched_pathways <- find_enriched_pathways_CAMERA(
            simulated_expression_filename,
            metadata_filename, 
            hallmark_DB_filename
        ) 
        write.table(as.data.frame(enriched_pathways), file = out_filename, row.names = F, sep = "\t")
        print("in CAMERA")
    }
    else if (enrichment_method == "ORA"){
        enriched_pathways <- find_enriched_pathways_ORA(
            simulated_expression_filename,
            metadata_filename, 
            hallmark_DB_filename
        ) 
        write.table(as.data.frame(enriched_pathways), file = out_filename, row.names = F, sep = "\t")
        print("in ORA")
    }
}

[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking sample ordering..."
[1] TRUE
[1] "in ROAST"
[1] "Checking 

## Format enrichment output

Each method yields a different output format so we will need to format the data before we can rank and summarize it

In [11]:
%%R -i hallmark_DB_filename -o hallmark_DB_names
library("GSA")

hallmark_DB <- GSA.read.gmt(hallmark_DB_filename)

hallmark_DB_names <- as.data.frame(hallmark_DB$geneset.names)

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Read 50 records

  warnings.warn(x, RRuntimeWarning)
/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Read 7424 items

  warnings.warn(x, RRuntimeWarning)


12345678910111213141516171819202122232425262728293031323334353637383940414243444546474849501
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [12]:
ranking.format_enrichment_output(
    local_dir, 
    project_id, 
    enrichment_method, 
    hallmark_DB_names,
    num_runs
)

## Rank pathways

In [13]:
analysis_type = "GSA"

template_GSEA_stats_filename = os.path.join(
    local_dir,
    "GSA_stats",
    f"{enrichment_method}_stats_template_data_{project_id}_real.txt"    
)
template_GSEA_stats, simulated_GSEA_summary_stats = ranking.process_and_rank_genes_pathways(
    template_GSEA_stats_filename,
    local_dir,
    num_runs,
    project_id,
    analysis_type,
    col_to_rank_pathways,
    enrichment_method
)

## Pathway summary table

In [14]:
# Create intermediate file: "<local_dir>/gene_summary_table_<col_to_rank_pathways>.tsv"
summary_pathway_ranks = ranking.generate_summary_table(
    template_GSEA_stats_filename,
    template_GSEA_stats,
    simulated_GSEA_summary_stats,
    col_to_rank_pathways,
    local_dir,
    'pathway',
    params
)

summary_pathway_ranks.sort_values(by="Z score", ascending=False).head()

/home/alexandra/anaconda3/envs/generic_expression/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,Pathway ID,Adj P-value (Real),Rank (Real),FDR (Real),Median adj p-value (simulated),Rank (simulated),Mean FDR (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
pathway,,,,,,,,,,
HALLMARK_SPERMATOGENESIS,HALLMARK_SPERMATOGENESIS,0.0001,25.5,0.0001,0.0001,25.5,0.000112,0.000060,25,-0.200000
HALLMARK_REACTIVE_OXIGEN_SPECIES_PATHWAY,HALLMARK_REACTIVE_OXIGEN_SPECIES_PATHWAY,0.0001,25.5,0.0001,0.0001,25.5,0.000104,0.000020,25,-0.200000
HALLMARK_TGF_BETA_SIGNALING,HALLMARK_TGF_BETA_SIGNALING,0.0001,25.5,0.0001,0.0001,25.5,0.000197,0.000425,25,-0.227416
HALLMARK_UV_RESPONSE_UP,HALLMARK_UV_RESPONSE_UP,0.0001,25.5,0.0001,0.0001,25.5,0.041571,0.162964,25,-0.254482
HALLMARK_TNFA_SIGNALING_VIA_NFKB,HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.0001,25.5,0.0001,0.0001,25.5,0.000959,0.003070,25,-0.279695


In [15]:
# Create `pathway_summary_filename`
summary_pathway_ranks.to_csv(pathway_summary_filename, sep='\t')